# Imputation, Outcome Measurement, and Inference 

## Introduction

What should you do when you encounter missing values in your data? Unfortunately, there is usually no *right* answer. However, different decisions on how to address missing values can have different implications on the inferences you draw from any analysis. One option is to omit observations that have missing values, but you may also choose to retain all observations and instead impute these missing values using various techniques. Imputation provides your best guess for each missing point's true value. Here, you will learn how to implement common imputation methods for missing data and how those methods affect outcome measures and inferences.

### Learning Objectives

* Explore options for imputing missing values

* Visualize estimate changes following imputation


In this notebook, you will focus on 2012-13 New Jersey graduates' earnings during their first year after graduation. The notebook provides demonstrations using the first quarter after graduation and you will replicate using the fourth quarter. Recall that in the [Data Exploration](2.Dataset_Exploration.ipynb) notebook, you initially examined the earnings distribution for all members of this cohort who had positive earnings in this time period in New Jersey. To evaluate the earnings outcomes of all 2012-13 New Jersey graduates, you need to decide what to do when you cannot find their earnings in the New Jersey Unemployment Insurance (UI) wage records. A person may not appear in New Jersey's UI wage records for several reasons:
- The person is unemployed. 
- The person is out of labor force, e.g., schooling, childcare, etc...
- The person was employed outside of New Jersey.
- The person's job is not covered in UI wage records, e.g.,self-employed, independent contractors, federal government works, etc. <a href='https://www.nap.edu/read/10206/chapter/11#294'>(Hotz and Scholz, 2002)</a>

You will explore the resulting earnings outcomes after applying different earnings imputation methods. The methods covered in this notebook include:
- Dropping all "missing" values
- Filling in zero for people who do not have records in New Jersey UI wage records data 
- Substituting missing values with the average earnings of people who are in the same degree fields and have the same gender
- Regression imputation

## R Setup and Database Connection

Before you begin, you need to run the code cells below to import the libraries and connect to the proper server.

In [ ]:
# Database interaction imports
library(odbc, warn.conflicts=F, quietly=T)

# For data manipulation/visualization
library(tidyverse, warn.conflicts=F, quietly=T)

# For faster date conversions
library(lubridate, warn.conflicts=F, quietly=T)

# Use percent() function
library(scales, warn.conflicts=F, quietly=T)

#### **Establish a Connection to the Server**

Now, we are ready to connect to the server. We will create the connection using the `DBI`  and `ODBC` libraries. 

> **Loading R libraries** and **establishing connection** should always be the first step in your Jupyter Notebooks. Make sure you copy these code chunks when you create a new notebook.

In [ ]:
# Connect to the server
con <- DBI::dbConnect(odbc::odbc(),
                     Driver = "SQL Server",
                     Server = "msssql01.c7bdq4o2yhxo.us-gov-west-1.rds.amazonaws.com",
                     Trusted_Connection = "True")

## Brief Manipulation: Isolating Earnings during first quarter after graduation

Before we start performing imputation, we need to do some quick data manipulation to isolate earnings from the first quarter after each individual's graduation. To do so, using the same approach as we did in the last [section](2.Data_Exploration.ipynb) of the Data Exploration notebook, we will create a new column, `quarter_number`, by taking the difference between `job_date` and `grad_date` then dividing by 13 and rounding to the nearest whole number.

In [ ]:
# read in earnings table
qry <- "
SELECT * 
FROM tr_nj_2021.dbo.nb_cohort_wages_link;
"

df_wages <- dbGetQuery(con, qry)


In [ ]:
# add in quarter number
df_wages <- df_wages %>%
    mutate(
        quarter_number = round(as.double(difftime(as.Date(job_date), as.Date(grad_date), units = "weeks")/13), 0)
    )

# see evidence
df_wages %>%
    select(grad_date, job_date, quarter_number) %>%
    head()

Now, we can simply create a data frame with just first quarter post-graduation wages using `filter()`.

In [ ]:
# Filter quarter 1 after graduation
q1_wages <- df_wages %>%
    filter(quarter_number == 1)


Because we will want to estimate the total earnings for each `hashed_ssn` in this quarter, not necessarily their wages per employer, let's aggregate `q1_wages` to find the total earnings for each member of this cohort in the entire quarter.

In [ ]:
# get total earnings in quarter
q1_wages <- q1_wages %>%
    group_by(hashed_ssn) %>%
    summarize(tot_wages = sum(wage)) %>%
    ungroup()

In [ ]:
# number of graduates with positive earnings
q1_num <- q1_wages %>%
    summarize(
        n=n_distinct(hashed_ssn)
    )

cat('The total graduates with positive earnings during their first quarter after graduation:', q1_num$n)

In [ ]:
# read in cohort
qry <- "
SELECT * 
FROM tr_nj_2021.dbo.nb_cohort_dated;
"

df <- dbGetQuery(con, qry)

In [ ]:
# read in major information
qry <- "
SELECT code_2010 as major, title_2010 as major_title
FROM ds_nj_oshe.dbo.supplements_cipcode;
"

df_cip <- dbGetQuery(con,qry)

df <- df %>% 
    left_join(df_cip, by.y = "major")

In [ ]:
cat('That is', percent(q1_num$n/nrow(df), .01), 'of the study cohort.')

### Checkpoint 1: Identifying Earnings in the Fourth Quarter after Graduation

Given the code above, create a data subset `q4_wages` that contains all earnings for the cohort in their fourth quarter after graduation. How many members of our cohort had positive earnings in this quarter? Do you expect this number to be higher or lower than the number in the first quarter?

In [ ]:
# replace __ in the code below with the appropriate quarter

q4_wages <- df_wages %>%
    filter(quarter_number == __)

# replace __ in the code below with the appropriate function
q4_wages <- q4_wages %>%
    ___(hashed_ssn) %>%
    summarize(tot_wages = sum(wage)) %>%
    ungroup()

# replace __ in the code below with the appropriate variable
q4_num <- __ %>%
    summarize(n=n_distinct(hashed_ssn))

cat('The total graduates with positive earnings during their fourth quarter after graduation:', q4_num$n)
cat('\nThat is', percent(q4_num$n/nrow(df), .01), 'of the study cohort.')

## Add graduates without positive earnings for Q1

Our current data frame, `q1_wages`, only contains individuals with positive earnings in their first quarter after graduation in New Jersey. Let's add in members of our cohort who did not appear in New Jersey's wage records during this time period, as well the additional variables from the original cohort table to better describe the individuals. This will let us easily analyze different earnings distributions in the cohort's first quarter after graduation as we progress throughout this notebook.

We can do so by using a `left_join()` of the original cohort, `df`, to `q1_wages`, as this will add in one row for each `coleridge_id` in the original cohort that was not included in `q1_wages`.

In [ ]:
# add grads without positive earnings
q1_all_wages <- df %>%
    left_join(q1_wages, c("hashed_ssn"))

As a quick check, we can see if the number of individuals in `q1_all_wages` that either have or do not have null wages makes sense given the total number of individuals in the cohort that were in `q1_wages`. We can do so by adding in an indicator variable if the `wages` column was null for each potential wage record in `q1_all_wages`, and then counting the number of distinct individuals based on this new variable.

In [ ]:
# employment outcomes for all of those in our original cohort
q1_all_wages %>%
    mutate(wage_ind = ifelse(is.na(tot_wages), 'no', 'yes')) %>%
    group_by(wage_ind) %>%
    summarize(n=n_distinct(hashed_ssn))

In [ ]:
q1_num$n

We can see that these numbers make sense. If they did not add up, chances are there was an issue with the details of your join.

Just to confirm, we can check to see if the number of rows in `q1_all_wages` is equal to the number of rows in `df`, the original cohort, as each individual in the original cohort should correspond to a single row regardless of employment status.

In [ ]:
nrow(df) == nrow(q1_all_wages)

Let's also check to see if we have any missing values for our demographic variables. If so, let's fill these in as `unknown` so they won't be dropped in future analyses.

In [ ]:
# see number of na values by column
colSums(is.na(q1_all_wages))

In [ ]:
q1_all_wages<-q1_all_wages %>%
    replace_na(list(
        hashed_njsmartid = 'U',
        hashed_instid ='U',
        birthyear = 'U',
        admstatus = 'U',
        yr_matriculation = 'U',
        sem_matriculation = 'U',
        accumulatedcredit = 'U',
        accumlatedgpa = 'U',
        hisp = 'U',
        race_ai_alask = 'U',
        race_as = 'U',
        race_aa = 'U',
        race_pi = 'U',
        race_wh = 'U',
        race_single = 'U'
    )
              )

# see na distribution now
colSums(is.na(q1_all_wages))

> Theoretically, you could apply these imputation methods to these missing demographic values. However, for the purposes of this notebook, we will focus our imputation techniques on missing earnings values.

### Checkpoint 2: Replicate for Q4 ###

Create a data frame `q4_all_wages` that mirrors `q1_all_wages` except for Q4. Feel free to add in as many code cells as you deem necessary.

In [ ]:
# replace __ in the code below with the appropriate word

q4_all_wages <- df %>%
   ___(q4_wages, c("hashed_ssn"))


In [ ]:
# see number of na values by column by replacing __ with the appropriate variable
colSums(is.na(___))

In [ ]:
# for each variable with missing data fill in with unknown, as shown above. 

q4_all_wages<-q4_all_wages %>%
    replace_na(list(
        ___ = 'U',
        ___ ='U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U',
        ___ = 'U'
    )
              )

# see na distribution now
colSums(is.na(q4_all_wages))

## Impute Wage Values

Now that we have confirmed that our `q1_all_wages` dataframe is ready to use for testing our imputation methods, we can get started. To recall, here are the four methods we will be trying out in this notebook:
1. Dropping all people with "missing" values on the variable of interest (Q1 wages)
2. Filling in zero for people who do not have records in New Jersey UI data
3. Filling in missing values with the average New Jersey UI earnings of people who are in the same degree fields and have the same gender
4. Regression

### 1. Drop All Missing Values

First, let's look at the earnings outcomes during first quarter after graduation when we drop all missing earnings values. Here, by ignoring potentially non-missing values, we are hoping that they mirror the same distribution as the present one. Although this is fairly common, you should **never, ever, ever** use this method in practice. 

> Deleting missing values is often called listwise deletion and essentially assumes that missing values are missing completely at random (MCAR). For a scholarly treatment of this issue, see (amongst others): 
> - Rubens (1976) "Inference and Missing Data" for the initial presentation, or
> - Peugh and Enders (2004) "Missing Data in Educational Research: A Review of Reporting Practices and Suggestions for Improvement" for a more recent discussion.  

In [ ]:
# drop missing values
q1_no_missing <- q1_all_wages %>% 
    filter(!is.na(tot_wages))

In [ ]:
# see earnings distribution
summary(q1_no_missing$tot_wages)

#### Checkpoint 3: Replicate for Q4

What does the earnings distribution look like for Q4 when you drop missing values?

In [ ]:
# replace __ with the appropriate variable

q4_no_missing <- ___ %>% 
    filter(!is.na(tot_wages))

summary(q4_no_missing$tot_wages)

### 2. Fill in Missing Values with Zero

Next, let's see how the earnings distribution shifts when we encode all missing earnings outcomes as 0. Here, we are assuming that all missing earnings are due to unemployment.

In [ ]:
# fill all null tot_wages with 0
q1_wages_zero <- q1_all_wages %>%
    mutate(tot_wages = ifelse(is.na(tot_wages), 0, tot_wages)) 

In [ ]:
# Take a look at the distribution. How does it vary from the distribution you get in method 1?
summary(q1_wages_zero$tot_wages)

In [ ]:
cat('Average earnings if missing wages are dropped is $', round(mean(q1_no_missing$tot_wages), 2), sep = '', '.')

cat('\nAverage earnings if missing wages are imputed as 0 is $', round(mean(q1_wages_zero$tot_wages), 2), sep = '', '.')

#### Checkpoint 4: Replicate for Q4 ####

What does the earnings distribution look like for Q4 when you fill missing values with zero?

In [ ]:
# replace __ with the appropriate function

q4_wages_zero <- q4_all_wages %>%
    mutate(tot_wages = ifelse(___(tot_wages), 0, tot_wages)) 
summary(q4_wages_zero$tot_wages)

cat('Average earnings if missing wages are dropped is $', round(mean(q4_no_missing$tot_wages), 2), sep = '', '.')

cat('\nAverage earnings if missing wages are imputed as 0 is $', round(mean(q4_wages_zero$tot_wages), 2), sep = '', '.')

### 3. Fill in Missing Values with Major/Gender Mean Earnings

Now, instead of either ignoring missing values or assuming the earnings are 0, we will try imputing missing earnings for each individual as the average quarterly earnings of the other individuals in our cohort of the same `sex` and `major_title`.

Here, our strategy is as follows:
- Using populated wages, find mean earnings for each major by gender
- Merge the mean earnings, based on major and gender, to the overall cohort
 - creates an additional column `mean_wages`
- Recode so that missing values are populated with mean earnings
 - data stored in a new column `imputed_wages`


>Note: This process is frequently termed mean imputation. Implementing this method will compress the variance and covariance of the imputed variable, resulting in biased parameter estimates for all parameters except the mean (Peugh & Enders, 2004, p.529). In this example, we are assuming that the missing values in wages are conditional on both gender and major. We also assume that the missingness is not truly indicative of lack of wages.

For the sex column, since group REDACTED has so few values, we will drop group REDACTED to create a binary variable and recode the remaining groups so Male = 1 and Female = 0.

In [ ]:
# filter out sex = 0 and recode
q1_all_wages <- q1_all_wages %>%
    filter(sex != 0) %>%
    mutate(
        sex = ifelse(sex == '1', '1', '0')
    )

In [ ]:
# mean earnings by gender/kpeds_major1 grouping
q1_all_wages %>%
    group_by(sex, major_title) %>%
    summarize(mean_wages = mean(tot_wages, na.rm=T)) %>%
    head()

In [ ]:
#mean earnings by gender/kpeds_major1 grouping saved
q1_major_gend <- q1_all_wages %>%
    group_by(sex, major_title) %>%
    summarize(mean_wages = mean(tot_wages, na.rm=T)) %>%
    ungroup()


Now, we will merge the two DataFrames, `q1_major_gend` and `q1_all_wages` using `inner_join`.
> Note: `left_join()` would also work in this case.

In [ ]:
# see if join works
q1_all_wages %>%
    inner_join(q1_major_gend, by=c('sex', 'major_title')) %>%
    head()

In [ ]:
# save join results to q1_joined_major_gend
q1_joined_major_gend <- q1_all_wages %>%
    inner_join(q1_major_gend, by=c('sex', 'major_title'))

Now, we can add a new column to `q1_joined_major_gend` to include the mean wage, based on gender and major, *if* the individual did not appear in the New Jersey UI wage records data. 

In [ ]:
# see if mutation works as designed
q1_joined_major_gend %>%
    mutate(imputed_wages = ifelse(is.na(tot_wages), mean_wages, tot_wages)) %>%
    select(tot_wages, mean_wages, imputed_wages) %>%
    head()

In [ ]:
# save mutation to q1_major_gend_impute
q1_major_gend_impute <- q1_joined_major_gend %>%
    mutate(imputed_wages = ifelse(is.na(tot_wages), mean_wages, tot_wages))

In using this method, there is a chance we cannot impute missing values for all individuals in the cohort. If `imputed_wages` is still `NA`, we can assume there were no individuals in the cohort with non-missing earnings with the same major/gender combination.

In [ ]:
# see if any still don't have imputed earnings
q1_major_gend_impute %>%
    filter(is.na(imputed_wages)) %>%
    summarize(n=n())

Unfortunately, it seems as though we do not have available earnings for every combination of gender and primary degree. For the sake of the exercise, we will ignore the earnings of those whose we could not impute using this method.

In [ ]:
# see distribution
summary(q1_major_gend_impute$imputed_wages)

In [ ]:
cat('Average earnings if missing wages are dropped is $', round(mean(q1_no_missing$tot_wages), 2), sep = '', '.')

cat('\nAverage earnings if missing wages are imputed as 0 is $', round(mean(q1_wages_zero$tot_wages), 2), sep = '', '.')

cat('\nAverage earnings if missing wages are imputed using major/gender means earnings is $', round(mean(q1_major_gend_impute$imputed_wages, na.rm=T), 2), sep = '', '.')

#### Checkpoint 5: Replicate for Q4 ####

Impute missing earnings values as the mean earnings of individuals in the cohort with the same gender (`gender`) and degree designation (`kpeds_major1`) in quarter 4. What does the earnings distribution look like? For how many individuals could you not impute values using this method?

In [ ]:
# fill in __ with appropriate variables 

# recode sex
q4_all_wages <- q4_all_wages %>%
    filter(sex != 0)

q4_all_wages <- q4_all_wages %>% 
    mutate(
        sex=ifelse(sex == 1, '1', '0')
    )

#mean earnings by gender/kpeds_major1 grouping saved
q4_major_gend <- q4_all_wages %>%
    group_by(___, ___) %>%
    summarize(mean_wages = mean(tot_wages, na.rm=T)) %>%
    ungroup()


# save join results to q1_joined_major_gend
q4_joined_major_gend <- q4_all_wages %>%
    inner_join(q4_major_gend, by=c('sex', 'major_title'))

# save mutation to q1_major_gend_impute
q4_major_gend_impute <- q4_joined_major_gend %>%
    mutate(imputed_wages = ifelse(is.na(tot_wages), mean_wages, tot_wages))



# see if any still don't have imputed earnings
q4_major_gend_impute %>%
    filter(is.na(imputed_wages)) %>%
    summarize(n=n())
q4_major_gend_impute <- q4_major_gend_impute %>% drop_na()

# see earnings distribution
summary(q4_major_gend_impute$imputed_wages)

cat('Average earnings if missing wages are dropped is $', round(mean(q4_no_missing$tot_wages), 2), sep = '', '.')

cat('\nAverage earnings if missing wages are imputed as 0 is $', round(mean(q4_wages_zero$tot_wages), 2), sep = '', '.')

cat('\nAverage earnings if missing wages are imputed using major/gender means earnings is $', round(mean(q4_major_gend_impute$imputed_wages), 2), sep = '', '.')

### 4. Regression imputation

We can also use regression to try to get more accurate earnings values. We will build a regression equation from the obervations for which we know the earnings, then use the equation to predict the missing earnings values. This is, in effect, an extension of the mean imputation by subgroup. Here, we will use demographic information of graduates such as birth year, sex, major, and time of graduation.

> Note: We will not be checking the assumptions associated with linear regressions, as this example is aimed at merely displaying how to use a linear regression for imputation. If you plan on using regression imputation, please check all assumptions before employing a predictive model.

In [ ]:
# subset to variables included in regression analysis
q1_reg <- q1_all_wages %>%
    select(hashed_ssn, tot_wages, birthyear, sex, major_title, grad_date)

To make sense of the linear regression, we will filter for the top 5 majors and then group all other majors together. 

In [ ]:
# finding top 5 majors
top_5_majors <- q1_reg %>%
    count(major_title) %>% 
    arrange(desc(n)) %>%
    head(5)

top_5_majors

In [ ]:
# creating an additional column with the top 5 majors and all others assigned to 6.

q1_reg <- q1_reg  %>%
    mutate(
        major_group = ifelse(q1_reg$major_title %in% top_5_majors$major_title, major_title, "Other")
    )
    

In [ ]:
# see variable types and outputs
q1_reg %>%
    glimpse()

In this case, it may make sense for `birthyear` to be a numeric variable rather than a character vector, as there may be some predictive power in numerically analyzing the ages of the graduates. Let's change `birthyear` to a `numeric` variable.

> As you saw above, there are a few individuals in the cohort with an unknown birthdate, so they will not be included in the predictive portion of this analysis.

In [ ]:
# convert birthyear to numeric
q1_reg <- q1_reg %>%
    mutate(
        birthyear = as.numeric(birthyear)
    )

In [ ]:
# don't need tot_wages because they are null 
q1_wages_na <- q1_reg %>%
    filter(is.na(tot_wages)) %>%
    select(-c(tot_wages))

# removing NAs from tot_wages
q1_wages_pred <- q1_reg %>%
    filter(!is.na(tot_wages))

The model creation process for a linear regression can be done using the `lm()` function. The variable we are trying to predict is on the left-hand side of `lm()` before the `~`, and the predictors are all of the variables on the right-hand side of the `~`.

In [ ]:
# run model and fit coefficients
q1_wages_model <- lm(tot_wages ~ birthyear + sex + major_group, data = q1_reg)

In [ ]:
# see model summary
summary(q1_wages_model)

Part of regression-based imputation is to evaluate your model for any unusual relationships. Examining the above result suggets that younger graduates tend to earn less, males tend to earn more, and business majors and registered nurses earn more than the comparison group (accounting majors) while the other major grouping earns less. While there is certainly more we could add to inform this model the sign of these coefficients make theoretical sense. 

Now that we have fit coefficients for each of the predictors in the model, we can predict the `tot_wages` variable for the missing data using `predict()`.

In [ ]:
# predict earnings
pred_earnings <- data.frame(tot_wages = predict(q1_wages_model, newdata=q1_wages_na))

In [ ]:
head(pred_earnings)

Because the output for `predict()` retains the same order of rows from `q1_wages_na`, we can add the `tot_wages` variable from `pred_earnings` into the existing `q1_wages_na` data frame.

In [ ]:
# see updated data frame with predicted earnings
cbind(q1_wages_na, pred_earnings) %>% 
    head()

In [ ]:
# save updated data frame with predicted earnings
q1_wages_na_w_earnings <- cbind(q1_wages_na, pred_earnings)

Finally, before we can see the effects of the imputation method, we need to combine our training set, which already has `tot_wages`, with our testing set and its predicted `tot_wages`. 

In [ ]:
# combine the q1_wages_na_w_earning and q1_wages_pred
rbind(q1_wages_na_w_earnings, q1_wages_pred) %>% 
    head()

In [ ]:
# save the combined dataframes
q1_reg_earnings <- rbind(q1_wages_na_w_earnings, q1_wages_pred)

Now we can see the entire earnings distribution for the cohort after applying regression imputation.

In [ ]:
# see earnings distribution for full cohort
summary(q1_reg_earnings$tot_wages)

In [ ]:
# see earnings distribution for imputed portion of cohort
summary(q1_wages_na_w_earnings$tot_wages)

## Visualizing Earnings Distributions

We can quickly determine if these different imputation methods significantly altered the pre-imputation wage distribution by visualizing the overall eranings distribution. Plotting side-by-side boxplots can be an effective choice. To do so, we need to bind the earnings from all of these methods by rows, meaning they must have the same columns. For the sake of simplicity, we will have three columns in this data frame:

- `hashed_ssn`, the person identifier
- `tot_wages`, cumulative earnings in first quarter post-graduation
- `method`, type of imputation method

In [ ]:
# adapt q1_no_missing
q1_no_missing %>%
    select(hashed_ssn, tot_wages) %>% head()

q1_no_missing <- q1_no_missing %>%
    select(hashed_ssn, tot_wages) %>%
    mutate(method = 'remove missing')

In [ ]:
# adapt q1_reg_earnings
q1_reg_earnings%>%
    select(hashed_ssn, tot_wages) %>% head()

q1_reg_earnings <- q1_reg_earnings %>%
    select(hashed_ssn, tot_wages) %>%
    mutate(method = 'regression')

In [ ]:
# adapt q1_wages_zero
q1_wages_zero %>%
    select(hashed_ssn, tot_wages) %>% head()

q1_wages_zero <- q1_wages_zero %>%
    select(hashed_ssn, tot_wages) %>%
    mutate(method = 'zero')

In [ ]:
#adapt q1_major_gend_impute
q1_major_gend_impute %>% select(hashed_ssn, imputed_wages) %>% rename(tot_wages = imputed_wages) %>% head()

q1_major_gend_impute <- q1_major_gend_impute %>%
    select(hashed_ssn, tot_wages) %>%
    mutate(method = 'mean')

Now that these methods all have the same column names, we can feed them into `rbind()`.

In [ ]:
# combine earnings from all methods
all_methods <- rbind(q1_major_gend_impute, q1_reg_earnings, q1_no_missing, q1_wages_zero)


Before visualizing these distributions, we will filter out extreme outliers that would affect the rest of the visualizations.

In [ ]:
# removing rows that have a tot_wages values greater then 50000

all_methods <- all_methods %>% 
    filter(tot_wages < 50000)

# check the max value
max(all_methods$tot_wages, na.rm=T)

In [ ]:
# boxplot of all methods
all_methods %>%
    ggplot(aes(x=method, y = tot_wages)) +
    geom_boxplot() + 
    labs(
        title = "The REDACTED Earnings Distribution's Quartiles are REDACTED REDACTED across \n imputation methods",
        x='Imputation Method',
        y='Quarter Earnings',
        caption = 'Source: NJ UI wage records data'
    ) +
    theme_minimal()

## Multiple histograms

We can also look at the differences in the earnings distribution by looking at side-by-side histograms. Instead of using the `geom_` layer `geom_boxplot()`, we will use `geom_histogram()`.

In [ ]:
all_methods %>%
    ggplot(aes(x=tot_wages)) +
    geom_histogram() + 
    facet_grid(method ~ .) +
    labs(
        title = 'Zero Imputation has a REDACTED REDACTED on the overall earnings distribution',
        y = 'Number of Workers',
        x='Quarterly Wages',
        caption = 'Source: NJ UI wage records data'
    ) +
    theme_minimal()

### (Optional) Advanced: Using machine learning to impute values

To impute values, we can also use machine learning algorithms such as `K-nearest Neighbors` and `Decision Trees`. The principle behind `K-nearest Neighbors` is quite simple: the missing values can be imputed by values of "closest neighbors" - as approximated by other, known, features. 

For example, if we had cases where the data on earnings of some graduates was completely missing, we could approximate their earnings by referring to other characteristics which could be shared by major group (their 'closest neighbors' in terms of characteristics).

The algorithm calculates the distance between the input values (the missing values) and helps to identify the nearest possible value based on other features (such as known characteristics of the closest major group). Imputing missing data using machine learning has become a research hotbed, and there are plenty of papers covering the various algorithms if you are curious.

## References

Peugh, J. L., & Enders, C. K. (2004). Missing Data in Educational Research: A Review of Reporting Practices and Suggestions for Improvement. _Review of Educational Research_, 74(4), 525-556. doi: 10.3102/00346543074004525

Rubin, D. B. (1976). Inference and Missing Data. _Biometrika_, 63(3), 581-592. doi:10.2307/2335739